In [1]:
import uproot
import awkward as ak
import numpy as np

In [2]:
klub_index_cols = ["EventNumber", "RunNumber", "lumi"]
klub_selection_cols = [
    "pairType",
    "nleps",
    "nbjetscand",
    "isLeptrigger",
    "bjet1_bID_deepFlavor",
    "bjet2_bID_deepFlavor",
    "isBoosted",
]
old_dnn_col = ['DNNoutSM_kl_1']

The file 'output_0_klub.root' was generated by running addBranch with the following 2 files:

KLUB file without dnn branch: /eos/user/l/lportale/hhbbtautau/skims/SKIMS_UL17/SKIM_ZZZ/output_0.root

dnn pred. file: /eos/user/j/jowulff/res_HH/Condor_out/predictions/30_10_23_param_allyears/2017/SKIM_ZZZ/output_0.root 

In [3]:
spin=0
mass=1000
model_name = '30_10_23_param_allyears'
branch_name = [f'dnn_{model_name}_spin{spin}_mass{mass}']

In [4]:
expressions = klub_index_cols+klub_selection_cols+list(old_dnn_col)+list(branch_name)

In [5]:
with uproot.open("./add_branch/output_0_klub.root") as f:
    data = f['HTauTauTree'].arrays(expressions = expressions)

In [11]:
baseline_mask = ((data.isLeptrigger == 1)
                & ((data.pairType == 0) | (data.pairType == 1) | (data.pairType == 2))
                & (data.nleps == 0) & (data.nbjetscand > 1))
            
data['pass_bl'] = baseline_mask

In [12]:
ak.sum(~data.pass_bl)

12346

In [13]:
len(data[data.DNNoutSM_kl_1<0])

12079

In [14]:
ak.sum(data.dnn_30_10_23_param_allyears_spin0_mass1000 < 0)

12346

In [15]:
12346-12079

267

In [16]:
neg_old_dnn = data[data.DNNoutSM_kl_1<0]
neg_old_dnn_selection = (neg_old_dnn.isLeptrigger == 1) & ((neg_old_dnn.pairType == 0) | (neg_old_dnn.pairType == 1) | (neg_old_dnn.pairType == 2)) & (neg_old_dnn.nleps == 0) & (neg_old_dnn.nbjetscand > 1)

In [17]:
np.unique(neg_old_dnn_selection)

<Array [False] type='1 * bool'>

In [18]:
pos_old_dnn = data[data.DNNoutSM_kl_1>=0]
pos_old_dnn_selection = (pos_old_dnn.isLeptrigger == 1) & ((pos_old_dnn.pairType == 0) | (pos_old_dnn.pairType == 1) | (pos_old_dnn.pairType == 2)) & (pos_old_dnn.nleps == 0) & (pos_old_dnn.nbjetscand > 1)

In [19]:
np.unique(pos_old_dnn_selection)

<Array [False, True] type='2 * bool'>

In [20]:
ak.sum(~pos_old_dnn_selection)

267

In [21]:
non_matching = data[(data.DNNoutSM_kl_1>=0) & ~(baseline_mask)]

In [22]:
ak.all(np.isin(non_matching[klub_selection_cols].pairType, [0,1,2]))

True

In [23]:
ak.all(non_matching[klub_selection_cols].nleps == 0)

True

In [24]:
ak.all(non_matching[klub_selection_cols].nbjetscand > 1)

True

In [25]:
ak.any(non_matching[klub_selection_cols].isLeptrigger == 1)

False